# Computing photometry and SEDs of galaxies in a diffsky mock

This notebook shows how to compute photometry through arbitrary bands of galaxies in a diffsky mock stored in the opencosmo format, and how to compute high-res SEDs.

## Loading the mock data

The next two cells cells show how to load the mock files that are natively produced by the diffsky source code. For purposes of this demo, we will just work with a redshift slice, which will be downloaded in the next cell.


In [ ]:
! wget -P ./download_dir -q -r -e robots=off -np -nH --cut-dirs=7 --reject "index.html*" https://portal.nersc.gov/project/hacc/aphearin/diffsky_data/smdpl_dr1_12_03_2025

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [1]:
from diffsky.data_loaders.opencosmo_utils import load_diffsky_mock

directory = "download_dir"

catalog, aux_data = load_diffsky_mock(directory)

NotADirectoryError: download_dir

### Inspect the mock data

The OpenCosmo toolkit provides extensive tooling for querying data, allowing you to select the specific types of objects you are interested in and prototype analyses on small samples before scaling them up. More details on the toolkit are available [in the documentation](https://opencosmo.readthedocs.io).

Additionally, the :code:`load_diffsky_mock` function returns auxilliary data which is used by diffsky to compute photometry or SEDs. The most important thing in this auxilliary data is the transmission curves, which determines which bands you can compute photometry for.

In [4]:
print("Columns:", catalog.columns)
print("Transmission curves:", aux_data["tcurves"]._fields)

NameError: name 'catalog' is not defined

### Recompute mock photometry

You can use the `compute_phot_from_diffsky_mock` function to compute mock photometry for any of the available bands.


In [ ]:
from diffsky.data_loaders.opencosmo_utils import compute_phot_from_diffsky_mock
computed_photometry = compute_phot_from_diffsky_mock(catalog, aux_data, z_phot_table, bands = ["lsst_u", "lsst_i"], insert = False)
exisiting_photometry = catalog.select(("lsst_u", "lsst_i")).get_data("numpy")

assert np.allclose(exisiting_photometry['lsst_u'], computed_photometry['lsst_u'], rtol=1e-4)
assert np.allclose(exisiting_photometry['lsst_i'], computed_photometry['lsst_i'], rtol=1e-4)

### Recompute photometry of disk/bulge/knot decomposition

The photometry of each morphological component can be calculated using the `compute_dbk_phot_from_diffsky_mock` function.

In [ ]:
from diffsky.data_loaders.opencosmo_utils import compute_dbk_phot_from_diffsky_mock
computed_dbk_photometry = compute_dbk_phot_from_diffsky_mock(catalog, aux_data, z_phot_table, bands = ["lsst_u", "lsst_i"], insert = False)
exisiting_bulge_photometry = catalog.select(("lsst_u_bulge", "lsst_i_bulge")).get_data("numpy")

assert np.allclose(exisiting_bulge_photometry['lsst_u_bulge'], computed_db_photometry['lsst_u_bulge'], rtol=1e-4)
assert np.allclose(exisiting_bulge_photometry['lsst_i_bulge'], computed_db_photometry['lsst_i_bulge'], rtol=1e-4)

### Compute SED of diffsky galaxies

There is also a convenience function for computing the high-resolution SED of each diffsky galaxy. This calculation is memory-intensive and requires more memory than allocated by readthedocs, and so the cell below is commented out, although most modern laptops can do this calculation for about 1000 galaxies at a time.

In [5]:
# from diffsky.data_loaders.opencosmo_utils import compute_sed_from_diffsky_mock 
# seds = compute_sed_from_diffsky_mock(catalog, aux_data, z_phot_table, bands = ["lsst_u", "lsst_i"], insert=False)

In [ ]:
# fig, ax = plt.subplots(1, 1)
# __=ax.loglog()
# xlim = ax.set_xlim(2_000, 200_000)
# ylim = ax.set_ylim(1e-8, 1e-3)

# igal = 50
# __=ax.plot(aux_data['ssp_data'].ssp_wave, seds["rest_sed"][igal, :])
# xlabel = ax.set_xlabel('wavelength [angstroms]')
# ylabel = ax.set_ylabel('Lsun/Hz')

### Compute SED of disk/bulge/knot components

There is an additional convenience function for computing the high-resolution SED of each morphological component.

In [ ]:
# from diffsky.data_loaders.opencosmo_utils import compute_dbk_sed_from_diffsky_mock 
# dbk_sed_info = compute_dbk_sed_from_diffsky_mock(catalog, aux_data, z_phot_table, bands = ["lsst_u", "lsst_i"], insert=False)

In [ ]:
# fig, ax = plt.subplots(1, 1)
# __=ax.loglog()
# xlim = ax.set_xlim(2_000, 200_000)
# ylim = ax.set_ylim(1e-8, 1e-4)

# igal = 50
# __=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_bulge'][igal, :])
# __=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_disk'][igal, :])
# __=ax.plot(diffsky_lc_patch['ssp_data'].ssp_wave, dbk_sed_info['rest_sed_knots'][igal, :])
# xlabel = ax.set_xlabel('wavelength [angstroms]')
# ylabel = ax.set_ylabel('Lsun/Hz')

### Computing photometry in other bands

You can compute photometry in other bandpasses by replacing the `tcurves` entry of `diffsky_lc_patch` with custom transmission curves.  

The next few cells show how to compute photometry through two fake bandpasses. We provide convinience functions for adding new transmission curves into the auxilliary data. This function expects tuples of (wavelength, tranmission_curve).

In [4]:
from jax.scipy.stats import norm as jnorm
from collections import namedtuple
from diffsky.data_loaders.opencosmo_utils import add_transmission_curves
wave = np.linspace(200, 8_000, 500)
fake_tcurve1 = jnorm.pdf(wave, loc=3_000.0, scale=500.0)
fake_tcurve2 = jnorm.pdf(wave, loc=5_000.0, scale=500.0)

aux_data = add_transmission_curves(aux_data, fake_tcurve1 = (wave, fake_tcurve1), fake_tcurve2 = (wave, fake_tcurve2))


__=ax.plot(fake_tcurve1.wave, fake_tcurve1.transmission)
__=ax.plot(fake_tcurve2.wave, fake_tcurve2.transmission)

# You should now see the new tranmission curves in the list of tranmission curves
print(aux_data["tcurves"]._fields)

SyntaxError: incomplete input (2900581736.py, line 15)

In [ ]:
Tcurves = namedtuple("Tcurves", ("fake_tcurve1", "fake_tcurve2"))
fake_tcurves = Tcurves(fake_tcurve1, fake_tcurve2)
computed_dbk_photometry = compute_dbk_phot_from_diffsky_mock(catalog, aux_data, z_phot_table, bands = ["fake_tcurve1", "fake_tcurve2"], insert = False)